# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 7 2023 10:41AM,258815,19,MSP219234,"Methapharm, Inc.",Released
1,Mar 7 2023 10:37AM,258814,19,ACG-2102493009,ACG North America LLC,Released
2,Mar 7 2023 10:33AM,258813,19,AVET3823,"Avet Pharmaceuticals Labs, Inc",Released
3,Mar 7 2023 10:27AM,258811,10,9236292,Eywa Pharma Inc.,Released
4,Mar 7 2023 10:27AM,258811,10,9238860,Eywa Pharma Inc.,Released
5,Mar 7 2023 10:27AM,258811,10,9238861,Eywa Pharma Inc.,Released
6,Mar 7 2023 10:27AM,258811,10,9238865,Eywa Pharma Inc.,Released
7,Mar 7 2023 10:27AM,258811,10,9238866,Eywa Pharma Inc.,Released
8,Mar 7 2023 10:27AM,258811,10,9238899,Eywa Pharma Inc.,Released
9,Mar 7 2023 10:27AM,258811,10,9239423,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,258809,Executing,1
23,258811,Released,15
24,258813,Released,1
25,258814,Released,1
26,258815,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258809,1.0,NaN
258811,NaN,15.0
258813,NaN,1.0
258814,NaN,1.0
258815,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258710,16.0,29.0
258715,1.0,0.0
258722,1.0,0.0
258750,1.0,0.0
258764,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258710,16,29
258715,1,0
258722,1,0
258750,1,0
258764,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258710,16,29
1,258715,1,0
2,258722,1,0
3,258750,1,0
4,258764,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258710,16,29
1,258715,1,
2,258722,1,
3,258750,1,
4,258764,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc."
1,Mar 7 2023 10:37AM,258814,19,ACG North America LLC
2,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc"
3,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.
18,Mar 7 2023 10:26AM,258803,10,Bio-PRF
25,Mar 7 2023 10:20AM,258809,12,Hush Hush
26,Mar 7 2023 10:03AM,258807,20,Else Nutrition
27,Mar 7 2023 9:46AM,258781,15,"Virtus Pharmaceuticals, LLC"
41,Mar 7 2023 9:45AM,258710,15,"Person & Covey, Inc."
86,Mar 7 2023 9:44AM,258804,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc.",,1
1,Mar 7 2023 10:37AM,258814,19,ACG North America LLC,,1
2,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc",,1
3,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.,,15
4,Mar 7 2023 10:26AM,258803,10,Bio-PRF,,7
5,Mar 7 2023 10:20AM,258809,12,Hush Hush,1,
6,Mar 7 2023 10:03AM,258807,20,Else Nutrition,,1
7,Mar 7 2023 9:46AM,258781,15,"Virtus Pharmaceuticals, LLC",,14
8,Mar 7 2023 9:45AM,258710,15,"Person & Covey, Inc.",16,29
9,Mar 7 2023 9:44AM,258804,10,"Citieffe, Inc.",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc.",1,
1,Mar 7 2023 10:37AM,258814,19,ACG North America LLC,1,
2,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc",1,
3,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.,15,
4,Mar 7 2023 10:26AM,258803,10,Bio-PRF,7,
5,Mar 7 2023 10:20AM,258809,12,Hush Hush,,1
6,Mar 7 2023 10:03AM,258807,20,Else Nutrition,1,
7,Mar 7 2023 9:46AM,258781,15,"Virtus Pharmaceuticals, LLC",14,
8,Mar 7 2023 9:45AM,258710,15,"Person & Covey, Inc.",29,16
9,Mar 7 2023 9:44AM,258804,10,"Citieffe, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc.",1,
1,Mar 7 2023 10:37AM,258814,19,ACG North America LLC,1,
2,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc",1,
3,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.,15,
4,Mar 7 2023 10:26AM,258803,10,Bio-PRF,7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc.",1.0,NaN
1,Mar 7 2023 10:37AM,258814,19,ACG North America LLC,1.0,NaN
2,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN
3,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.,15.0,NaN
4,Mar 7 2023 10:26AM,258803,10,Bio-PRF,7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc.",1.0,0.0
1,Mar 7 2023 10:37AM,258814,19,ACG North America LLC,1.0,0.0
2,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0
3,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.,15.0,0.0
4,Mar 7 2023 10:26AM,258803,10,Bio-PRF,7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2329100,42.0,4.0
12,517601,2.0,5.0
15,776255,43.0,17.0
16,258750,0.0,1.0
19,1552828,21.0,5.0
20,517529,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2329100,42.0,4.0
1,12,517601,2.0,5.0
2,15,776255,43.0,17.0
3,16,258750,0.0,1.0
4,19,1552828,21.0,5.0
5,20,517529,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,42.0,4.0
1,12,2.0,5.0
2,15,43.0,17.0
3,16,0.0,1.0
4,19,21.0,5.0
5,20,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,2.0
2,15,Released,43.0
3,16,Released,0.0
4,19,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,4.0,5.0,17.0,1.0,5.0,1.0
Released,42.0,2.0,43.0,0.0,21.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,4.0,5.0,17.0,1.0,5.0,1.0
1,Released,42.0,2.0,43.0,0.0,21.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,4.0,5.0,17.0,1.0,5.0,1.0
1,Released,42.0,2.0,43.0,0.0,21.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()